In [1]:
import os, glob, subprocess, datetime

import pandas as pd
import numpy as np
import pickle as pkl
import scipy.stats as stats
import properscoring as ps
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from scores import *
from sklearn.neighbors import KernelDensity
from scipy.integrate import quad
from scipy import interpolate
from ipywidgets import *
from sklearn.linear_model import BayesianRidge, LinearRegression
from scipy.stats import multivariate_normal, norm
from scipy.interpolate import make_smoothing_spline

path_to_fPCA   = '/Users/Guille/Desktop/dynamic_update/software/fPCA'
path_to_fDepth = '/Users/Guille/Desktop/dynamic_update/software/fDepth'
path_to_data   = '/Users/Guille/Desktop/dynamic_update/data'

# Load and Processing Actual Generation

In [6]:
# Resource
resource = 'wind'

# Asset index
i_asset = 10

# Time Zone
T_zone = 24

ac_ = pd.read_csv(path_to_data + '/actuals/' + resource + '_actual_5min_site_2017.csv')
print(ac_.columns[i_asset + 1])

dates_    = ac_[['Time']].to_numpy()[T_zone*12:]
ac_       = ac_[[ac_.columns[i_asset + 1]]].to_numpy()[T_zone*12:]
F_ac_tr_  = ac_.reshape(int(ac_.shape[0]/288), 288)[:-1, :]
dates_tr_ = dates_.reshape(int(dates_.shape[0]/288), 288)[:-1, :]
print(F_ac_tr_.shape, dates_tr_.shape)

ac_ = pd.read_csv(path_to_data + '/actuals/' + resource + '_actual_5min_site_2018.csv')
print(ac_.columns[i_asset + 1])

dates_    = ac_[['Time']].to_numpy()[T_zone*12:]
ac_       = ac_[[ac_.columns[i_asset + 1]]].to_numpy()[T_zone*12:]
F_ac_ts_  = ac_.reshape(int(ac_.shape[0]/288), 288)[:-1, :]
dates_ts_ = dates_.reshape(int(dates_.shape[0]/288), 288)[:-1, :]
print(F_ac_ts_.shape, dates_ts_.shape)

Baffin
(363, 288) (363, 288)
Baffin
(363, 288) (363, 288)


# Load and Processing day-ahead forecast

In [8]:
# Time Zone
T_zone = 17

fc_ = pd.read_csv(path_to_data + '/actuals/' + resource + '_day_ahead_forecast_2018.csv')
print(fc_.columns[i_asset + 3])

fc_ = fc_[[fc_.columns[i_asset + 3]]].to_numpy()[T_zone:-(24 - T_zone)][:, 0]
print(fc_.shape)

x_ac_ = np.linspace(0, ac_.shape[0] - 1, ac_.shape[0], dtype = int)[:-11]
x_fc_ = np.linspace(0, fc_.shape[0] - 1, fc_.shape[0], dtype = int)*12
print(x_ac_.shape)
print(x_fc_.shape)

fc_ts_   = interpolate.interp1d(x_fc_, fc_, kind = 'linear')(x_ac_)
fc_ts_   = fc_ts_[:-277]
F_fc_ts_ = fc_ts_.reshape(int(fc_ts_.shape[0]/288), 288)
print(F_fc_ts_.shape)

Baffin
(8736,)
(104821,)
(8736,)
(363, 288)


In [287]:
# Enfore max and min
def _gen_constraint(f_, f_min, f_max):
    f_[f_ < f_min] = f_min
    f_[f_ > f_max] = f_max
    return f_

# Define update function
def _update_rate(N, update_rate = 1):
    x_ = np.linspace(0, N - 1, N)
    w_ = np.exp(-x_/(update_rate*1e4 + 1e-6))
    return w_

# Define forget function
def _forget_rate(N, forget_rate = 1):
    x_ = np.linspace(0, N - 1, N)
    w_ = np.exp(-x_*forget_rate)
    return w_[::-1]

# Calculate weighted (w_) distance between X_ and x_
def _dist(X_, x_, w_):
    d_ = np.zeros((X_.shape[0], ))
    for i in range(X_.shape[0]):
        d_[i] = w_.T @ (X_[i, :] - x_)**2
    return d_

# Radial Basis function kernel based on distance (d_)
def _kernel(d_, length_scale):
    w_ = np.exp(-d_/length_scale)
    return w_/w_.sum()

def _smoothing(F_, f_, lamdba):
    x_        = np.linspace(0, f_.shape[0] - 1, f_.shape[0])
    F_smooth_ = F_.copy()
    f_smooth_ = make_smoothing_spline(x_, f_, lam = lamdba)(x_)
    for i in range(F_.shape[0]):
        F_smooth_[i, :] = make_smoothing_spline(x_, F_[i, :], lam = lamdba)(x_)
    F_ = F_smooth_.copy()
    f_ = f_smooth_.copy()
    
    return F_, f_

# Fuse day-ahead forecast with real-time forecast
def _update_forecast(F_ac_, f_hat_, fc_, update_rate):

    eta_      = _update_rate(F_ac_.shape[1] + 1, update_rate = update_rate)
    w_update_ = np.nan_to_num((eta_/eta_.max())[1:], 0.)[::-1]
    f_update_ = f_hat_*(1. - w_update_) + fc_*w_update_

    # plt.figure(figsize = (10, 2))
    # plt.title('Trust Rate')
    # plt.plot(w_update_)
    # plt.ylim(-0.1,1.1)
    # plt.show()

    return f_update_

# Multivariate normal forecast assumption
def _predictive_multivariate_normal_dist(_model, fc_):

    F_ = _model['F_ts_']
    w_ = _model['weights']  
    
    # Mean function
    f_hat_ = F_.T @ w_ 

    # Regulate mean function
    f_hat_ = _gen_constraint(f_hat_, _model['f_min'], _model['f_max'])
    
    # Smoothing prediction: unobserved mean and actuals
    if (_model['smoothing'] == 2) | (_model['smoothing'] == 3): 
        F_, f_hat_ = _smoothing(F_, f_hat_, _model['lambda'])
        
    # Fuse day-ahead forecast with real-time forecast
    mu_hat_ = _update_forecast(F_, f_hat_, fc_, _model['trust_rate'])

    # Covariance function
    F_hat_ = np.repeat(mu_hat_[:, np.newaxis], F_.shape[0], axis = 1).T
    S_hat_ = (F_ - F_hat_).T @ np.diag(w_) @ (F_ - F_hat_)
    
    # plt.figure(figsize = (10, 2))
    # plt.plot(f_hat_, label = 'real-time (fc)')
    # plt.plot(fc_, label = 'day-ahead (fc)')
    # plt.plot(mu_hat_, label = 'update (fc)')
    # plt.ylim(-0.1,)
    # plt.legend()
    # plt.show()

    # Define probability dist
    _N = multivariate_normal(mu_hat_, S_hat_, allow_singular = True)
    
    _model['normal'] = {}
    _model['normal']['mean']         = mu_hat_
    _model['normal']['covariance']   = S_hat_
    _model['normal']['distribution'] = _N
    
    return _model.copy()

In [289]:
# Define functional forecast model and functional observations
def _ffc_fit(F_, t_event, forget_rate  = 1e6,
                          trust_rate   = 0,
                          length_scale = 100, 
                          lamdba       = 1, 
                          smoothing    = 0,
                          tr_kNNs      = 0.01,
                          n_kNNs       = 0,
                          n_min_kNNs   = 5,
                          n_max_kNNs   = 50,
                          uniform_kNNs = False):
    
    _model = {}
    
    _model['f_min'] = ac_tr_.min()
    _model['f_max'] = ac_tr_.max()
    _model['F_tr_'] = ac_tr_[:, :t_event]
    _model['F_ts_'] = ac_tr_[:, t_event:]
    
    # Forget rate:
    # 0 - weight equal to the all past obervation time series 
    # 1 - full weight on the most recent observations
    _model['forget_rate'] = forget_rate
    
    # Thrust rate to day-ahead forecast:
    # 0 - Don't trust the day-head forecast
    # 1 - Trust the day-ahead forecast fully
    _model['trust_rate'] = trust_rate
    
    # Kernel based distance lenght-scale parameter:
    # lenght-scale: smaller lenght-scale weights heavier closest samples
    _model['length_scale'] = length_scale

    # Number of k nearest neighbors (n_kNNs or tr_kNNs)
    # n_kNNs: Force n to be k
    # tr_kNNs: Adaptive thershold by weight probability
    # * k_min: minimim number of neightbors when applied tr_kNNs
    # * k_max: maximum number of neightbors when applied tr_kNNs
    # uniform_kNNs: True selected samples weights uniform, False distance based kernel wegihts
    _model['n_kNNs']       = n_kNNs
    _model['tr_kNNs']      = tr_kNNs
    _model['n_min_kNNs']   = n_min_kNNs
    _model['n_max_kNNs']   = n_max_kNNs
    _model['uniform_kNNs'] = uniform_kNNs
    
    # Smoothing functions:
    # 0: Don't apply smoothing 
    # 1: Smooth observations
    # 2: Smooth prediction
    # 3: Smooth observations and prediction
    # * lamdba: smoothing parameter 0 less smooth 
    _model['smoothing'] = smoothing
    _model['lamdba']    = lamdba

    # # Number of random scenarios 
    # _model['n_scens'] = n_scens

    return _model

# Forecast dynamic update based on functional kNNs
def _fknn_fc_predict(_model, f_, fc_):
    
    F_ = _model['F_tr_']
    
    # Smoothing observed mean and actuals
    if (_model['smoothing'] == 1) | (_model['smoothing'] == 3): 
        F_, f_ = _smoothing(F_, f_, _model['lambda'])

    # phi: importance weights based on time distance
    phi_ = _forget_rate(f_.shape[0], _model['forget_rate'])
    
    # plt.figure(figsize = (10, 2))
    # plt.title('Forget Rate')
    # plt.plot(phi_)
    # plt.ylim(-0.1,1.1)
    # plt.show()

    # d: euclidian distance between samples weighted by importance weights (phi)
    d_ = _dist(F_, f_, phi_/phi_.sum())
    # w: normalized wieghts distance across observations based on RBF kernel distance
    w_ = _kernel(d_, _model['length_scale'])
        
    # If no k-Nearest Neibors use adaptative approach
    if _model['n_kNNs'] == 0:
        # Find the kNNs 
        i    = int(np.sum(np.cumsum(np.sort(w_)[::-1]) < _model['tr_kNNs']))
        idx_nn_ = w_ > np.sort(w_)[::-1][i]
        # Enforce upper (max) and lower (min) Nearest Neibors limit
        if (idx_nn_.sum() < _model['n_min_kNNs']):
            idx_nn_ = w_ > np.sort(w_)[-_model['n_min_kNNs'] - 1]
        if (idx_nn_.sum() > _model['n_max_kNNs']):
            idx_nn_ = w_ > np.sort(w_)[-_model['n_max_kNNs'] - 1]      
    else:
        k = _model['n_kNNs'].copy() + 1
        # Find the kNNs 
        idx_nn_ = w_.argsort().argsort() > w_.shape[0] - k
        # Get equivalent threshold
        tr_kNNs = w_[idx_nn_].min()

    # Calculate kNNs weights
    w_knn_ = np.zeros((idx_nn_.shape[0],))
    if _model['uniform_kNNs']:
        w_knn_[idx_nn_] = 1.
    else:
        w_knn_[idx_nn_] = w_[idx_nn_]
    _model['weights'] = w_knn_/w_knn_.sum()

    # Multivariate normal assumption
    _model = _predictive_multivariate_normal_dist(_model, fc_)
    
    return _model
    
# Forecast dynamic update based on distance
def _ffc_predict(_model, f_, fc_):
    
    F_ = _model['F_tr_']
    
    # Smoothing observed mean and actuals
    if (_model['smoothing'] == 1) | (_model['smoothing'] == 3): 
        F_, f_ = _smoothing(F_, f_, _model['lambda'])

    # phi: importance weights based on time distance
    phi_ = _forget_rate(f_.shape[0], _model['forget_rate'])
    
    # plt.figure(figsize = (10, 2))
    # plt.title('Forget Rate')
    # plt.plot(phi_)
    # plt.ylim(-0.1,1.1)
    # plt.show()

    # d: euclidian distance between samples weighted by importance weights (phi)
    d_ = _dist(F_, f_, phi_/phi_.sum())
    # w: normalized wieghts distance across observations based on RBF kernel distance
    w_ = _kernel(d_, _model['length_scale'])

    _model['weights'] = w_/w_.sum()

    # Multivariate normal assumption
    _model = _predictive_multivariate_normal_dist(_model, fc_)

    return _model

# Select a test sample and forecast and split into traning and test
def _get_test_sample(F_, F_fc_ts_, i_day, t_event):

    f_    = F_[i_day, :]
    f_tr_ = F_[i_day, :t_event]
    f_ts_ = F_[i_day, t_event:]
    #print(f_.shape, f_tr_.shape, f_ts_.shape)

    fc_    = F_fc_ts_[i_day, :]
    fc_tr_ = F_fc_ts_[i_day, :t_event]
    fc_ts_ = F_fc_ts_[i_day, t_event:]
    #print(fct_.shape, fct_tr_.shape, fct_ts_.shape)

    return f_tr_, fc_tr_, f_ts_, fc_ts_

In [290]:
yearday  = 50
interval = 140

f_tr_, fc_tr_, f_ts_, fc_ts_ = _get_test_sample(F_ac_ts_, F_fc_ts_, yearday, interval)

_model = _ffc_fit(F_ac_tr_, interval,
                  forget_rate  = 1,
                  trust_rate   = 0,
                  length_scale = 100, 
                  lamdba       = 1, 
                  smoothing    = 0,
                  tr_kNNs      = 0.01,
                  n_kNNs       = 0,
                  n_min_kNNs   = 5,
                  n_max_kNNs   = 50,
                  uniform_kNNs = False)

_model_fknn = _fknn_fc_predict(_model, f_tr_, fc_ts_)
_model_fcc  = _ffc_predict(_model, f_tr_, fc_ts_)